<a href="https://colab.research.google.com/github/AugustinCablant/Viager/blob/main/code_commune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [63]:
!jupyter nbconvert --to pdf code commune.ipynb

[NbConvertApp] WARNING | pattern 'code' matched no files
[NbConvertApp] WARNING | pattern 'commune.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equiva

In [62]:
!pip install nbconvert

In [1]:
!pip install reportlab
!pip install pdfkit
!pip install tqdm
!pip install dateparser
!pip install unidecode

import pandas as pd
import numpy as np
from datetime import datetime
import locale
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table
import pdfkit
import tabulate
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import html
import dateparser
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Stage')
from tqdm import tqdm
from unidecode import unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 55.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.9 MB/s eta 0:00:00
Mounted at /content/drive


# Code

In [54]:
df = pd.read_csv('v_commune_depuis_1943.csv')
df.columns

Index(['COM', 'TNCC', 'NCC', 'NCCENR', 'LIBELLE', 'DATE_DEBUT', 'DATE_FIN'], dtype='object')

**Description des colonnes** : <br>
- La colonne 'COM' correspond au Code commune <br>
- La colonne 'TNCC' correspond au Type de nom en clair <br>
- La colonne 'NCC' correspond au Nom en clair (majuscules) <br>
- La colonne 'NCCENR' correspond au Nom en clair (typographie riche) <br>
- La colonne 'LIBELLE' correspond au Nom en clair (typographie riche) avec article <br>
- La colonne 'DATE_DEBUT' correspond au Date de début du couple code*libellé de commune (AAAA-MM-JJ) <br>
- La colonne 'DATE_FIN' correspond au Date de fin du couple code*libellé de commune (AAAA-MM-JJ). Lorsque cette colonne est vide, cela signifie que la commune est toujours active. <br>

In [55]:
# Commencer par faire un DATAFRAME des communes qui ont été définitivement supprimées

df2 = df.copy() # DATAFRAME sans les communes qui ont été définitivement supprimées
supp = pd.DataFrame(columns = ['nom_commune','code_commune'],index = df.index.to_list())

k = 0
for i in tqdm(df.index.to_list()):
   if pd.notnull(df['DATE_FIN'][i]): # Cette ville n'existe plus
    autres = df[df['COM']==df['COM'][i]]
    #print(autres.shape[0])
    if autres.shape[0]==1:
      supp['nom_commune'][k] = df['NCC'][i]
      supp['code_commune'][k] = df['COM'][i]
      df2 = df2.drop(i)
      k += 1

    elif autres.shape[0]>1:
      autres = autres.drop(i)
      indices = autres.index.to_list()
      for l in indices:
        if pd.notnull(autres['DATE_FIN'][l]):
          supp['nom_commune'][k] = df['NCC'][l]
          supp['code_commune'][k] = df['COM'][l]
          k +=1
          if l in df2.index.to_list():
            df2 = df2.drop(l)

supp = supp.dropna()

100%|██████████| 42198/42198 [01:22<00:00, 509.64it/s] 


In [56]:
print("Il y a ",supp.shape[0], " communes qui ont été supprimées. En voici un apperçu :")
print(supp.head(10))

Il y a  4845  communes qui ont été supprimées. En voici un apperçu :
                nom_commune code_commune
0                  AMAREINS        01003
1                     ARLOD        01018
2                    ARNANS        01020
3  BELLEGARDE SUR VALSERINE        01033
4                BELLEGARDE        01033
5         BELMONT LUTHEZIEU        01036
6                   BELMONT        01036
7                      BEON        01039
8                     BOHAS        01048
9                   BOUVENT        01055


In [60]:
# Établir un DATAFRAME des communes ayant changé de nom et étant encore existante

transformation = pd.DataFrame(columns = ['ancien_nom','nouveau_nom','code_commune'],index = df.index.to_list())
k = 0
for i in tqdm(df2.index.to_list()):
  if pd.notnull(df2['DATE_FIN'][i]): # Cette ville n'existe plus

    # Trouver le nouveau nom :
    nouveaux_noms = df2[df2['COM']==df2['COM'][i]]
    nouveaux_noms = nouveaux_noms.drop(i)
    # ! Il peut y en avoir plusieurs
    # C'est le cas où il y a plusieurs changements
    # Après avoir fait tourné un code simple, on apprend qu'il y a 474 cas
    # Il nous suffit de récupérer la liste des anciens noms et de mettre le nouveau nom

    nv = nouveaux_noms[nouveaux_noms['DATE_FIN'].isnull()]['NCC'].to_list()[0]
    for j in nouveaux_noms.index.to_list():
      transformation['ancien_nom'][k] = nouveaux_noms['NCC'][j]
      transformation['code_commune'][k] = nouveaux_noms['COM'][j]
      transformation['nouveau_nom'][k] = nv
      k +=1

transformation = transformation.dropna()

100%|██████████| 37415/37415 [00:17<00:00, 2150.23it/s]


In [58]:
print("Il y a ",transformation.shape[0], " communes dont le nom a changé.")


Il y a  2470  communes dont le nom a changé.


In [61]:
transformation.to_csv('liste_transformations_communes.csv')
supp.to_csv('liste_suppression_communes.csv')